In [5]:
from env import get_db_url
from datetime import datetime
import pandas as pd
import numpy as np
from pydataset import data

## Exercises I

In [3]:
# Shown here for exercise completion. Actual function is defined in env.py and imported above (per lesson)
def get_db_url(user, hostname, password, database_name):
    return f'mysql+pymysql://{user}:{password}@{hostname}/{database_name}'

In [7]:
url = get_db_url('employees')

In [8]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [ ]:
# Intentionally make typo in database url
url = url + '332'
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

In [9]:
# The url typo results in an OperationalError and Access denied

In [ ]:
# Intentionally making error in SQL query
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFFSET 50', url)

In [15]:
# That typo in the SQL query resulted in a Programming Error

In [10]:
employees = pd.read_sql('SELECT * FROM employees', url)
titles = pd.read_sql('SELECT * FROM titles', url)

7. How many rows and columns for each DataFrame

In [11]:
# Rows and columns for each dataframe matches SQL database
employees.shape 

(300024, 6)

In [12]:
titles.shape

(443308, 4)

In [20]:
# Row and column numbers match the SQL database (checked using Sequel-Ace)

8. Summary statistics for each dataframe

In [28]:
employees.describe(include = 'all')

,emp_no,birth_date,first_name,last_name,gender,hire_date
count,300024.000000,300024,300024,300024,300024,300024
unique,NaN,4750,1275,1637,2,5434
top,NaN,1952-03-08,Shahab,Baba,M,1985-06-20
freq,NaN,95,295,226,179973,132
mean,253321.763392,NaN,NaN,NaN,NaN,NaN
std,161828.235540,NaN,NaN,NaN,NaN,NaN
min,10001.000000,NaN,NaN,NaN,NaN,NaN
25%,85006.750000,NaN,NaN,NaN,NaN,NaN
50%,249987.500000,NaN,NaN,NaN,NaN,NaN
75%,424993.250000,NaN,NaN,NaN,NaN,NaN


In [29]:
titles.describe(include = 'all')

,emp_no,title,from_date,to_date
count,443308.000000,443308,443308,443308
unique,NaN,7,6393,5888
top,NaN,Engineer,1998-10-25,9999-01-01
freq,NaN,115003,132,240124
mean,253075.034430,NaN,NaN,NaN
std,161853.292613,NaN,NaN,NaN
min,10001.000000,NaN,NaN,NaN
25%,84855.750000,NaN,NaN,NaN
50%,249847.500000,NaN,NaN,NaN
75%,424891.250000,NaN,NaN,NaN


9. How many unique titles in titles DataFrame

In [35]:
# We can see 7 unique titles as show above using describe(). Also can generate list of the titles and number as follows:

print(f"There are {len(titles.title.unique())} unique title(s) in the titles DataFrame")

There are 7 unique title(s) in the titles DataFrame


10. Oldest date in the to_date column

In [38]:
print(f"The oldest date in the to_date column for the titles table is {titles.to_date.min()}")

The oldest date in the to_date column for the titles table is 1985-03-01


11. Most recent date in the to_date column

In [66]:
print(f"The most recent date in the to_date column for the titles table (besides {titles.to_date.max()}) is {titles.to_date[titles.to_date <=datetime.today().date()].max()}")

The most recent date in the to_date column for the titles table (besides 9999-01-01) is 2002-08-01


## Exercises II

1. Copy the users and roles DataFrames from the examples above.

In [53]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [54]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


What is the result of using a right join on the DataFrames?

In [55]:
users.merge(roles, left_on = 'role_id', right_on = 'id', how = "right")

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


What is the result of using an outer join on the DataFrames?

In [57]:
users.merge(roles,left_on = 'role_id', right_on = 'id', how = "outer")

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [81]:
users.drop(columns='role_id').merge(roles.drop(columns = 'id'))

,id,name


Load the mpg dataset from PyDataset.

In [13]:
mpg = data('mpg')

Output and read the documentation for the mpg dataset.

In [90]:
data('mpg',show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [58]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


How many rows and columns are in the dataset?

In [96]:
print(f'There are {mpg.shape[0]} rows and {mpg.shape[1]} columns in the mpg dataset')

There are 234 rows and 11 columns in the mpg dataset


Check out your column names and perform any cleanup you may want on them.

In [14]:
mpg = mpg.rename(columns = {'cty':'city_mpg','hwy':'highway_mpg','drv':'drive'})

In [15]:
mpg.describe(include='all')

,manufacturer,model,displ,year,cyl,trans,drive,city_mpg,highway_mpg,fl,class
count,234,234,234.000000,234.000000,234.000000,234,234,234.000000,234.000000,234,234
unique,15,38,NaN,NaN,NaN,10,3,NaN,NaN,5,7
top,dodge,caravan 2wd,NaN,NaN,NaN,auto(l4),f,NaN,NaN,r,suv
freq,37,11,NaN,NaN,NaN,83,106,NaN,NaN,168,62
mean,NaN,NaN,3.471795,2003.500000,5.888889,NaN,NaN,16.858974,23.440171,NaN,NaN
std,NaN,NaN,1.291959,4.509646,1.611534,NaN,NaN,4.255946,5.954643,NaN,NaN
min,NaN,NaN,1.600000,1999.000000,4.000000,NaN,NaN,9.000000,12.000000,NaN,NaN
25%,NaN,NaN,2.400000,1999.000000,4.000000,NaN,NaN,14.000000,18.000000,NaN,NaN
50%,NaN,NaN,3.300000,2003.500000,6.000000,NaN,NaN,17.000000,24.000000,NaN,NaN
75%,NaN,NaN,4.600000,2008.000000,8.000000,NaN,NaN,19.000000,27.000000,NaN,NaN


How many different manufacturers are there?

In [16]:
print(f'There are {len(mpg.manufacturer.unique())} unique manufacturers in this dataset')

There are 15 unique manufacturers in this dataset


How many different models are there?

In [17]:
print(f'There are {len(mpg.model.unique())} unique models in this dataset')

There are 38 unique models in this dataset


Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [18]:
mpg['mileage_difference'] = mpg.highway_mpg - mpg.city_mpg

Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

In [19]:
mpg['average_mileage'] = mpg[['highway_mpg','city_mpg']].mean(axis = 1)

Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

In [21]:
mpg['is_automatic'] = mpg.trans.str.contains('auto')

Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [33]:
mpg.groupby('manufacturer').average_mileage.agg('mean').nlargest(1,keep = 'all')

manufacturer
honda    28.5
Name: average_mileage, dtype: float64

Do automatic or manual cars have better miles per gallon?

In [43]:
grouping_mpg_trans = mpg.groupby('is_automatic').average_mileage.agg('mean')
grouping_mpg_trans

is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

In [42]:
if ~grouping_mpg_trans.idxmax():
    print("Manual cars have better average miles per gallon")
else:
    print("Automatic cars have better mileage per gallon")

Manual cars have better average miles per gallon


## Exercises III

Use your get_db_url function to help you explore the data from the chipotle database.

In [ ]:
chipotle_url = get_db_url(user, host, password, 'chipotle')

In [46]:
pd.read_sql('SHOW TABLES', get_db_url('chipotle'))

,Tables_in_chipotle
0,orders


In [47]:
orders = pd.read_sql('SELECT * FROM orders', get_db_url('chipotle'))

What is the total price for each order?

In [48]:
# Convert the item_price column to float to allow for aggregation
orders.item_price = orders.item_price.str.replace('$','').astype(float)
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  4622 non-null   int64  
 1   order_id            4622 non-null   int64  
 2   quantity            4622 non-null   int64  
 3   item_name           4622 non-null   object 
 4   choice_description  4622 non-null   object 
 5   item_price          4622 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 216.8+ KB


/var/folders/tt/99v8dp0d60sfw82fs51dhr4m0000gn/T/ipykernel_53337/584713939.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  orders.item_price = orders.item_price.str.replace('$','').astype(float)


In [49]:
orders.groupby('order_id').item_price.agg('sum')

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64

What are the most popular 3 items?

In [50]:
print(f"The 3 most popular items are: {list(orders.groupby('item_name').quantity.agg('sum').nlargest(3, keep = 'all').index)}")

The 3 most popular items are: ['Chicken Bowl', 'Chicken Burrito', 'Chips and Guacamole']


Which item has produced the most revenue?

In [ ]:
# Create an item revenue column which is the product of the quantity ordered and the item price
#orders['item_revenue'] = orders.quantity * orders.item_price

In [51]:
orders.groupby('item_name').item_price.agg('sum').nlargest(1, keep = "all")

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

Join the employees and titles DataFrames together.

In [14]:
emp_t = employees.merge(titles)

For each title, find the hire date of the employee that was hired most recently with that title.

In [60]:
emp_t.groupby('title').hire_date.agg('max')

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [19]:
sql = """
SHOW TABLES
"""

In [21]:
employees_url = get_db_url('employees')

In [22]:
pd.read_sql(sql,employees_url)

,Tables_in_employees
0,departments
1,dept_emp
2,dept_manager
3,employees
4,salaries
5,titles


In [23]:
sql = """
SELECT * FROM departments
"""

In [24]:
departments = pd.read_sql(sql,employees_url)

In [25]:
sql = """
SELECT * FROM dept_emp
"""

In [26]:
dept_emp = pd.read_sql(sql,employees_url)

In [28]:
all_df = emp_t.merge(dept_emp).merge(departments)

In [29]:
all_df

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date,dept_no,dept_name
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,d005,Development
1,10006,1953-04-20,Anneke,Preusig,F,1989-06-02,Senior Engineer,1990-08-05,9999-01-01,d005,Development
2,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15,Assistant Engineer,1998-03-11,2000-07-31,d005,Development
3,10014,1956-02-12,Berni,Genin,M,1987-03-11,Engineer,1993-12-29,9999-01-01,d005,Development
4,10021,1960-02-20,Ramzi,Erde,M,1988-02-10,Technique Leader,1988-02-10,2002-07-15,d005,Development
...,...,...,...,...,...,...,...,...,...,...,...
142895,499868,1956-11-14,Shaz,Bierbaum,M,1988-11-01,Staff,1996-10-03,9999-01-01,d002,Finance
142896,499873,1960-12-10,Kasidit,Picel,F,1986-04-21,Staff,1997-08-03,9999-01-01,d002,Finance
142897,499950,1964-07-15,Weidon,Gente,F,1991-06-05,Staff,1999-02-02,2000-02-23,d002,Finance
142898,499977,1956-06-05,Martial,Weisert,F,1996-09-17,Staff,1999-12-28,9999-01-01,d002,Finance


In [32]:
# This includes historical data (for departments and employees)
pd.crosstab(all_df.title, all_df.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,1,2666,0,0,0,2044,531,0,0
Engineer,0,18311,0,0,0,14733,3682,5,0
Senior Engineer,2,13331,0,0,0,10748,2745,3,0
Senior Staff,3032,1,2893,2909,2969,0,0,2942,9074
Staff,4602,1,4434,4704,4551,1,0,4602,13813
Technique Leader,0,6802,0,0,0,5447,1321,0,0


In [61]:
# The following will filter for current employees in the department using existing dfs
current_df = all_df[all_df.to_date == all_df.to_date.max()]
pd.crosstab(current_df.title, current_df.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,0,1631,0,0,0,1275,317,0,0
Engineer,0,11342,0,0,0,9117,2317,2,0
Senior Engineer,1,10601,0,0,0,8607,2198,1,0
Senior Staff,2475,0,2296,2348,2374,0,0,2316,7333
Staff,2862,0,2724,2916,2850,0,0,2869,8592
Technique Leader,0,5414,0,0,0,4334,1044,0,0
